<a href="https://colab.research.google.com/github/DemianVL18/Analisis-de-Datos/blob/main/Implementando_PuLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N9: Optimización Lineal III: Formulación de problemas**

### **Introducción**

 - En los notebooks anteriores de esta unidad aprendiste los conceptos fundamentales para formular un problema de optimización lineal y solucionarlo con python. Esta semana el foco estará en poner a prueba estos aprendizajes.
  - Es por esto que el notebook de esta semana contiene información sobre dos desafíos empresariales, para los cuales deberás formular y responder el problema implementando PuLP. La idea es que las horas de estudio de la semana estén en la integración de los conocimientos previos sobre estos casos aplicados. Así, para cada caso deberán:
    - Caracterizar conceptual y matemáticamente:
      - Objetivo
      - Función objetivo
      - Variables
      - Restricciones
    - Solucionar el problema con PuLP

### **Caso 1: Planificación de rutas de distribución**


- Una empresa busca disminuir los costos totales de transporte al asignar la cantidad óptima de productos a tres ubicaciones diferentes.

- Estas ubicaciones se identifican como Ubicación 1, Ubicación 2 y Ubicación 3, y cada una tiene sus propias demandas y costos asociados.

- Es posible combinar distintas variables como la cantidad de producto a transportar  y la ubicación.

- Sin embargo, la suma total de los productos transportados a todas las ubicaciones no puede superar las 100 unidades (esta restricción garantiza que no se sobrecargue la capacidad de transporte disponible).

- Asimismo, para cumplir con las necesidades de cada ubicación, se establecen requisitos mínimos de demanda: se requiere un mínimo de 20 unidades en la Ubicación 1; se requiere un mínimo de 30 unidades en la Ubicación 2; se requiere un mínimo de 25 unidades en la Ubicación 3.

EN EL CASO 1, FALTA AGREGAR ESTOS ANTECEDENTES:

Costos por unidad para cada ubicación:

Costo por unidad transportada a Ubicación 1 es 40 USD.

Costo por unidad transportada a Ubicación 2 es 30 USD.

Costo por unidad transportada a Ubicación 3 es 50 USD.

**Elementos a considerar en la Respuesta 1:**

**- Objetivo:** Disminuir los costos totales de transporte.

**- Variables:**
 *   $ \text{X}_{1}  = \text{Cantidad de productos transportados a Ubicación 1}$
 *   $ \text{X}_{2}  = \text{Cantidad de productos transportados a Ubicación 2}$
 *   $ \text{X}_{3}  = \text{Cantidad de productos transportados a Ubicación 3}$

**- Función objetivo:**

* $ \text{Minimizar Costo Total} = \text{40X}_{1} + \text{30X}_{2} + \text{50X}_{3}  $

**- Restricciones:**
- Capacidad de trasporte:
 - $\text{X}_{1} + \text{X}_{2} + \text{X}_{3} \leq 100 $
- Demanda minima por ubicacion:
 - $ \text{X}_{1} \geq \text{20}$
 - $ \text{X}_{2} \geq \text{30}$
 - $ \text{X}_{3} \geq \text{25}$

In [ ]:
!pip install pulp
from pulp import *

In [ ]:
!pip install pulp
from pulp import *

# Definir los datos del problema
costos = {
    'Ubicacion1': 40,
    'Ubicacion2': 30,
    'Ubicacion3': 50,
}

capacidad_total = 100  # Capacidad máxima de transporte

# Crear el problema de minimización
problem = LpProblem("Planificación_de_Rutas", LpMinimize)

# Definir variables de decisión
# X1, X2 y X3 representan las cantidades transportadas a cada ubicación
variables = LpVariable.dicts("X", ['Ubicacion1', 'Ubicacion2', 'Ubicacion3'], lowBound=0, cat='Continuous')

# Definir la función objetivo
# Minimizar el costo total de transporte
problem += lpSum(costos[ubicacion] * variables[ubicacion] for ubicacion in costos)

# Restricción de capacidad total de transporte
problem += lpSum(variables[ubicacion] for ubicacion in variables) == capacidad_total, "CapacidadTotal"

# Restricciones de demanda mínima por ubicación
problem += variables['Ubicacion1'] >= 20, "DemandaMin_Ubicacion1"
problem += variables['Ubicacion2'] >= 30, "DemandaMin_Ubicacion2"
problem += variables['Ubicacion3'] >= 25, "DemandaMin_Ubicacion3"

# Resolver el problema
problem.solve()

# Mostrar los resultados
print("Estado de la solución:", LpStatus[problem.status])
print("Costo total de transporte:", value(problem.objective))
print("Cantidad de productos transportados a cada ubicación:")
for ubicacion in variables:
    print(f"{ubicacion}: {value(variables[ubicacion])} unidades")

Estado de la solución: Optimal
Costo total de transporte: 3700.0
Cantidad de productos transportados a cada ubicación:
Ubicacion1: 20.0 unidades
Ubicacion2: 55.0 unidades
Ubicacion3: 25.0 unidades


**Análisis:**

La solución muestra que para minimizar los costos totales de transporte, la estrategia óptima es transportar únicamente las cantidades mínimas requeridas por cada ubicación: 20 unidades a Ubicación 1, 30 a Ubicación 2 y 25 a Ubicación 3 (75 unidades). Esta asignación cumple la restricción de capacidad máxima permitida de 100 unidades y cumple estrictamente con los requerimientos de cada ubicación.

Este resultado es coherente con el objetivo de minimizar costos, ya que:

 * Enviar unidades adicionales a cualquier ubicación implicaría un aumento innecesario en el gasto, especialmente hacia Ubicación 3, cuyo costo por unidad es el más alto (50).
 * El modelo prioriza las ubicaciones con menor costo unitario (como Ubicación 2, con 30 por unidad), pero está limitado por la capacidad total de transporte, lo que impide cubrir más allá de la demanda mínima sin sacrificar el costo total.

En otras palabras, no hay beneficio económico en exceder las demandas mínimas, y hacerlo solo incrementaría el costo sin mejorar la eficiencia. Por tanto, el modelo opta por una asignación exacta y ajustada al mínimo exigido en cada destino, maximizando así la eficiencia del transporte bajo una restricción presupuestaria y de capacidad estricta.

Ahora bien, si la empresa decidiera llenar completamente la capacidad máxima del camión (100 unidades), el modelo propone esta alternativa óptima:

*	Ubicación 1: 20 unidades
*	Ubicación 2: 55 unidades
*	Ubicación 3: 25 unidades
*	Costo total de transporte: 3700

Este escenario permite aprovechar al máximo la capacidad logística disponible, pero con un incremento en el costo total de 750 respecto al escenario base. Esto demuestra que llenar el camión no siempre es sinónimo de eficiencia económica, y que la minimización de costos puede estar en tensión con la utilización total de recursos físicos.


### **Caso 2. Maximización de Ingresos por ventas**

- Imagina que ocupas el cargo de gerente de ventas en una empresa que ofrece una variedad de productos a sus clientes.

- La empresa cuenta con un portafolio diversificado de productos, cada uno con su propio potencial de generación de ingresos.

- La empresa busca aumentar su rentabilidad maximizando los ingresos por ventas. Esto implica tomar decisiones estratégicas sobre cómo asignar los recursos de ventas de manera eficiente para satisfacer la demanda del mercado y aprovechar al máximo la capacidad de producción.

- Se venden tres tipos de producto, para los cuales se tiene la cantidad vendida: producto_1, producto_2, y producto_3.

- Se asume que el precio de venta por unidad es de $20 para el producto 1, $15 para el producto 2 y $25 para el producto 3.

- La cantidad total de productos vendidos no puede exceder la capacidad de producción de la empresa (500). La demanda del mercado para cada producto debe ser satisfecha.

- Se establecen requisitos mínimos de demanda: 100 para el producto 1; 150 para el producto 2; y 200 para el producto 3.

- El desafío consiste en encontrar la asignación óptima de la cantidad de productos a vender que maximice los ingresos totales por ventas, garantizando que se cumplan todas las restricciones establecidas.

- Una vez resuelto, se obtendrá la cantidad óptima de cada producto a vender y los ingresos totales máximos por ventas.

**Elementos a considerar en la Respuesta 2:**

**- Objetivo:** Aumentar su rentabilidad maximizando los ingresos por ventas.

**- Variables:**
 *   $ \text{P}_{1}  = \text{Cantidad vendida Producto 1}$
 *   $ \text{P}_{2}  = \text{Cantidad vendida Producto 2}$
 *   $ \text{P}_{3}  = \text{Cantidad vendida Producto 3}$

**- Función objetivo:**
 - $ \text{Maximizar Ingresos} = \text{20P}_{1} + \text{15P}_{2} + \text{25P}_{3}  $

**- Restricciones:**
- Capacidad de produccion:
 - $\text{P}_{1} + \text{P}_{2} + \text{P}_{3} \leq 500 $
- Demanda minima por Producto:
 - $ \text{P}_{1} \geq \text{100}$
 - $ \text{P}_{2} \geq \text{150}$
 - $ \text{P}_{3} \geq \text{200}$

In [ ]:
from pulp import *

# Crear un problema de maximizando
problem = LpProblem("Maximizar_Alcance", LpMaximize)

# Definir variables
P1 = LpVariable("Producto 1", lowBound=0)
P2 = LpVariable("Producto 2", lowBound=0)
P3 = LpVariable("Producto 3", lowBound=0)

# Definir función objetivo: costo total de transporte (expresión que define la función objetivo)
problem += 20*P1 + 15*P2 + 25*P3

# Restricciones (expresiones que definen las restricciones)
problem += P1 + P2 + P3 <= 500
problem += P1 >= 100
problem += P2 >= 150
problem += P3 >= 200

# Resolver el problema
problem.solve()

# Método para resolver el problema y encontrar la solución óptima

# Mostrar resultados
print("Cantidad óptima de Producto 1 a vender:", value(P1))
print("Cantidad óptima de Producto 2 a vender:", value(P2))
print("Cantidad óptima de Producto 3 a vender:", value(P3))
print("Ingresos totales máximos por ventas:", value(problem.objective))


Cantidad óptima de Producto 1 a vender: 100.0
Cantidad óptima de Producto 2 a vender: 150.0
Cantidad óptima de Producto 3 a vender: 250.0
Ingresos totales máximos por ventas: 10500.0


**Análisis:**

Los resultados indican que la asignación óptima de productos a vender para maximizar los ingresos está compuesta por:

*	100 unidades de Producto 1
*	150 unidades de Producto 2
*	250 unidades de Producto 3

Esta asignación cumple exactamente con los mínimos de demanda establecidos (100, 150 y 200 respectivamente) y utiliza el total de la capacidad de producción disponible (500 unidades), lo que evidencia un uso eficiente y completo de los recursos productivos.

El modelo prioriza la venta del Producto 3, que tiene el mayor precio por unidad (25), maximizando su participación hasta agotar la capacidad de producción disponible, una vez satisfechos los mínimos exigidos para los otros productos. Esta estrategia permite alcanzar un ingreso total máximo de $10,500, que es el mejor resultado posible bajo las restricciones planteadas.

En términos económicos, el modelo asigna recursos donde generan el mayor retorno por unidad vendida, ajustando los productos de menor precio (Producto 1 y 2) al mínimo requerido. Esto demuestra una planificación estratégica orientada a la rentabilidad, donde se aprovecha al máximo el producto más rentable sin incumplir con las exigencias del mercado.
